In [ ]:
import pandas as pd
import os
from datetime import date, datetime, timedelta
from pathlib import Path
from dotenv import load_dotenv
from sqlalchemy import create_engine
from Mods import pandas_mod as pdm
from Mods import date_mod as dtm

In [ ]:
def reassign_id(df, id_col_name, id_str):
    """根據原有最後一筆資料進行自動編號"""

    # 先找出原本的id編號（id欄位非空）最大值
    nums = df.loc[df[id_col_name] != "", id_col_name].str.extract(r"(\d+)").astype(int)

    if nums.empty:
        start_num = 1
    else:
        start_num = nums.max()[0] + 1

    # 計算需要新增的資料數
    empty_id = df[id_col_name] == ""
    empty_id_count = empty_id.sum()

    # 先列出編號list
    new_id = [f"{id_str}{i:03d}" for i in range(start_num, start_num + empty_id_count)]

    # 將list放入df欄位
    df.loc[empty_id, id_col_name] = new_id

    return df

In [3]:
# 連線資料庫，並讀取主檔
load_dotenv()

username = os.getenv("MYSQL_USERNAME")
password = os.getenv("MYSQL_PASSWORD")
target_ip = os.getenv("MYSQL_IP")
target_port = int(os.getenv("MYSQL_PORTT"))
db_name = os.getenv("MYSQL_DB_NAME")

engine = create_engine(f"mysql+pymysql://{username}:{password}@{target_ip}:{target_port}/{db_name}")

sql = "SELECT * FROM airport"

df_main = pd.read_sql(sql, engine)

In [ ]:
# 設定欄位名、檔案路徑
folder = r"C:\Users\add41\Documents\Data_Engineer\Project\Flights-Data-Crawler\Data"

corp_list = ["EVA", "CAL", "SJX", "TTW"]
columns = ["airport", "code_1", "code_2"]
day = dtm.get_yesterday()

# 根據公司開始迴圈，逐一讀入並合併至主表
for corp in corp_list:
    file = f"{day}_{corp}_FlightList.csv"
    exist, file_path = pdm.exist_or_not(folder, file)

    # 若找不到今日資料檔案則回傳訊息並跳過
    if exist:
        df = pd.read_csv(file_path)
    else:
        print(f"查無{corp}的{day}資料，請確認資料是否存在！")
        continue

    # 將起飛和抵達機場做成兩個df並合併，最後再與主表合併
    depar_airport = df[["departure_airport", "departure_airport_code_1", "departure_airport_code_2"]]
    arrival_airport = df[["arrival_airport", "arrival_airport_code_1", "arrival_airport_code_2"]]

    depar_airport.columns = columns
    arrival_airport.columns = columns

    df_airport = pd.concat([depar_airport, arrival_airport], ignore_index=True)
    df_airport["APT_id"] = ""

    df_main = pd.concat([df_main, df_airport], ignore_index=True)

查無EVA的2025-10-27資料，請確認資料是否存在！
查無CAL的2025-10-27資料，請確認資料是否存在！
查無SJX的2025-10-27資料，請確認資料是否存在！
查無TTW的2025-10-27資料，請確認資料是否存在！


In [5]:
# 去除重複資料及含有空值資料（理論上不應存在，如有則去除）
subset_col = ["airport", "code_1", "code_2"]
df_main.drop_duplicates(subset=subset_col, inplace=True, keep="first")
df_main.dropna(inplace=True)

# 透過函式自動編號
df_main = reassign_id(df_main, "APT_id", "APT")

# 存檔至地端
main_file = "airport.csv"
main_path = os.path.join(folder, main_file)
df_main.to_csv(main_path, index=False)

# 存檔至資料庫
df_main.to_sql(name="airport", con=engine, index=False, if_exists="replace")

print(f"已更新機場列表，目前資料筆數：{len(df_main)}")

已更新機場列表，目前資料筆數：54
